# 3.5 - The Council of Eight: Multi-Agent SINKT Densification

Este notebook implementa a arquitetura **Multi-Agente (Swarm)** para densificação e validação do grafo de conhecimento SINKT. 

Substituindo as simulações anteriores, utilizamos o **LangGraph** para orquestrar 8 agentes autônomos especializados, cada um utilizando um modelo de IA alocado estrategicamente de acordo com a complexidade da tarefa.

## 🏛️ O Conselho (The Council)

| Agente | Função | Modelo Alvo |
| :--- | :--- | :--- |
| **1. The Scout** | Geração massiva de candidatos (Busca Vetorial). | `text-embedding-3-small` |
| **2. The Bridge** | **Anti-Ilhas**: Conexões entre capítulos distantes. | `gpt-5.1` |
| **3. The Professor** | Validação Pedagógica (Pré-requisitos). | `gpt-5.1` |
| **4. The Engineer** | Validação Técnica (Precisão Linux). | `claude-opus-4-5` |
| **5. The Ontologist** | Classificação Semântica (Taxonomia). | `gpt-4.1` |
| **6. The Topologist** | Proteção de DAG (Detecção de Ciclos). | `gpt-4.1` + *Python Tools* |
| **7. The Cleaner** | Remoção de Ruído e Alucinações. | `gpt-4.1-mini` |
| **8. The Judge** | Veredito Final e Resolução de Conflitos. | `gpt-5.1` |

In [7]:
# Instalação do LangGraph e dependências
!pip install -q langgraph langchain langchain-openai langchain-anthropic networkx scikit-learn numpy pydantic tqdm python-dotenv

import os
import json
import numpy as np
import networkx as nx
from typing import List, Literal, Optional, Dict, Any
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

# --- CONFIGURAÇÃO DE MODELOS (TIERED COMPUTE) ---
def get_model(model_name: str, temperature: float = 0):
    # Fallback seguro para modelos existentes
    if "claude" in model_name:
        return ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=temperature)
    elif "gpt-4o-mini" in model_name:
         return ChatOpenAI(model="gpt-4o-mini", temperature=temperature)
    else:
        # Padrão robusto para tarefas complexas
        return ChatOpenAI(model="gpt-4o", temperature=temperature)

# Registro de Modelos do Conselho (4 Agentes Otimizados)
MODELS = {
    "scout_embed": OpenAIEmbeddings(model="text-embedding-3-small"),
    "cleaner": get_model("gpt-4o-mini"),  # Triagem Rápida e Barata
    "expert": get_model("gpt-4o"),        # Validação Técnica e Pedagógica Profunda
    "analyst": get_model("gpt-4o"),       # Estrutura e Ontologia
    "judge": get_model("gpt-4o"),         # Síntese e Decisão
}

print("✅ Modelos Configurados e Carregados (Conselho de 4 Agentes).")

✅ Modelos Configurados e Carregados (Conselho de 4 Agentes).


In [8]:
# --- SETUP DE DIRETÓRIOS ---
INPUT_FOLDER = "output/01_extraction"
OUTPUT_FOLDER = "output/03_council_execution"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

CONCEPTS_FILE = f"{INPUT_FOLDER}/concepts_map.json"
RELATIONS_FILE = f"{INPUT_FOLDER}/relations_initial.json"

# Carregar Dados Iniciais
def load_data():
    with open(CONCEPTS_FILE, 'r', encoding='utf-8') as f:
        concepts = json.load(f)
    with open(RELATIONS_FILE, 'r', encoding='utf-8') as f:
        relations = json.load(f)
    return concepts, relations

concepts, initial_relations = load_data()

# Remover ruídos prévios (se houver)
concepts = [c for c in concepts if c.get('tipo') != 'NOISE']

print(f"📊 Dados Carregados: {len(concepts)} conceitos | {len(initial_relations)} relações iniciais.")

📊 Dados Carregados: 216 conceitos | 207 relações iniciais.


## 1. Fase de Geração de Candidatos (Scout & Bridge)

Aqui combinamos duas estratégias:
1.  **Scout:** Similaridade de Cosseno com threshold relaxado (0.75) para capturar muitas possibilidades.
2.  **Bridge:** Lógica explícita para forçar comparações entre conceitos de capítulos distantes (evitar ilhas).

In [9]:
def generate_candidates(concepts, relations_existing):
    print("\n🕵️ Scout & Bridge iniciando varredura...")
    
    # 1. Preparar Grafo Existente (para não sugerir duplicatas)
    G_exist = nx.DiGraph()
    for r in relations_existing:
        G_exist.add_edge(r['source'], r['target'])
        
    # 2. Gerar Embeddings
    texts = [f"{c['nome']} ({c['tipo']}): {c['definicao']} [Cap {c['capitulo_origem']}]" for c in concepts]
    vectors = MODELS['scout_embed'].embed_documents(texts)
    matrix = np.array(vectors)
    sim_matrix = cosine_similarity(matrix)
    
    candidates = []
    
    # Parâmetros Scout
    THRESHOLD_SCOUT = 0.75
    
    # Parâmetros Bridge (Anti-Ilha)
    # Se a distância entre capítulos for > 3, aceitamos um threshold menor para encorajar conexões longas
    THRESHOLD_BRIDGE = 0.70
    
    count_scout = 0
    count_bridge = 0
    
    for i in range(len(concepts)):
        source = concepts[i]
        # Extrair primeiro capítulo como referência (simplificação)
        src_chap = int(source['capitulo_origem'].split(',')[0]) if source['capitulo_origem'][0].isdigit() else 0
        
        for j in range(len(concepts)):
            if i == j: continue
            
            target = concepts[j]
            tgt_chap = int(target['capitulo_origem'].split(',')[0]) if target['capitulo_origem'][0].isdigit() else 0
            
            # Verificar se já existe
            if G_exist.has_edge(source['nome'], target['nome']) or G_exist.has_edge(target['nome'], source['nome']):
                continue
                
            score = sim_matrix[i][j]
            chap_dist = abs(src_chap - tgt_chap)
            
            # Lógica de Seleção
            category = None
            
            # BRIDGE: Distância alta e score razoável
            if chap_dist >= 3 and score > THRESHOLD_BRIDGE:
                category = 'BRIDGE_CANDIDATE'
                count_bridge += 1
            
            # SCOUT: Score alto independente da distância
            elif score > THRESHOLD_SCOUT:
                category = 'SCOUT_CANDIDATE'
                count_scout += 1
                
            if category:
                candidates.append({
                    "source": source['nome'],
                    "target": target['nome'],
                    "source_type": source['tipo'],
                    "target_type": target['tipo'],
                    "score": float(score),
                    "origin": category,
                    "chapter_dist": chap_dist
                })
                
    # Deduplicar (A->B e B->A podem ter scores idênticos, manter apenas um par para o Conselho decidir a direção)
    # Mas aqui deixaremos ambos pois a direção importa para o embeddings as vezes? Não, cosseno é simétrico.
    # Vamos filtrar para não superlotar o conselho: Top 300 candidatos mais fortes + Top 100 Bridges
    
    bridges = sorted([c for c in candidates if c['origin'] == 'BRIDGE_CANDIDATE'], key=lambda x: x['score'], reverse=True)[:150]
    scouts = sorted([c for c in candidates if c['origin'] == 'SCOUT_CANDIDATE'], key=lambda x: x['score'], reverse=True)[:350]
    
    final_candidates = bridges + scouts
    
    print(f"✅ Candidatos Gerados: {len(final_candidates)}")
    print(f"   - Via Bridge (Anti-Ilha): {len(bridges)} (de {count_bridge} encontrados)")
    print(f"   - Via Scout (Similaridade): {len(scouts)} (de {count_scout} encontrados)")
    
    return final_candidates

# Executar Geração
candidates = generate_candidates(concepts, initial_relations)


🕵️ Scout & Bridge iniciando varredura...


✅ Candidatos Gerados: 252
   - Via Bridge (Anti-Ilha): 150 (de 156 encontrados)
   - Via Scout (Similaridade): 102 (de 102 encontrados)


## 2. Definição do LangGraph (O Conselho)

Definimos o `GraphState` que carrega o dossiê da aresta e os nós para cada agente.

In [10]:
# --- ESTADO DO GRAFO ---
class AgentVote(BaseModel):
    agent: str
    verdict: Literal['APPROVE', 'REJECT', 'MODIFY', 'ABSTAIN']
    suggested_type: Optional[str] = None
    reason: str

class EdgeState(BaseModel):
    source: str
    target: str
    source_type: str
    target_type: str
    current_type: Optional[str] = "RELATED_TO" # Tipo proposto inicialmente
    similarity_score: float
    origin: str # Bridge ou Scout
    
    # Votos dos Agentes
    votes: List[AgentVote] = []
    
    # Decisão Final
    final_verdict: Optional[Literal['KEEP', 'DISCARD']] = None
    final_type: Optional[str] = None
    final_reason: Optional[str] = None

# --- AGENTES OTIMIZADOS --- 

# 1. THE CLEANER (Triagem Rápida - GPT-4o-Mini)
def agent_cleaner(state: EdgeState):
    prompt = ChatPromptTemplate.from_template(
        """Você é o CLEANER do grafo de conhecimento. Sua função é proteger o sistema de lixo e alucinações.
        
        Analise a aresta proposta:
        {source} ({source_type}) -> {target} ({target_type})
        Score Vetorial: {score}
        
        ### CRITÉRIOS DE ELIMINAÇÃO IMEDIATA (Vote REJECT):
        1. **Alucinações:** Conceitos que não existem no contexto Linux (ex: "Página 12", "Capítulo 4", "José Silva").
        2. **Tipos Errados:** Variáveis soltas (ex: "$PATH") conectadas a conceitos teóricos sem sentido.
        3. **Meta-dados:** Conexões com artefatos do livro (ex: "Figura 1.1", "Tabela 2").
        
        Se a conexão for minimamente plausível tecnicamente, vote ABSTAIN para deixar os especialistas decidirem.
        Seja conservador na aprovação, mas agressivo na eliminação de lixo óbvio.
        """
    )
    
    response = MODELS['cleaner'].invoke(prompt.format(
        source=state.source, source_type=state.source_type,
        target=state.target, target_type=state.target_type,
        score=state.similarity_score
    ))
    
    verdict = 'REJECT' if 'REJECT' in response.content.upper() else 'ABSTAIN'
    state.votes.append(AgentVote(agent="Cleaner", verdict=verdict, reason=response.content[:150]))
    return state

# 2. THE EXPERT (Técnico & Pedagogo - GPT-4o)
def agent_expert(state: EdgeState):
    # Se Cleaner rejeitou, não gasta token caro
    if any(v.verdict == 'REJECT' for v in state.votes): return state
    
    prompt = ChatPromptTemplate.from_template(
        """Você é o ESPECIALISTA SÊNIOR (Engenheiro Linux + Pedagogo).
        Analise a relação: {source} -> {target}.
        
        ### SUAS DUAS MISSÕES:
        1. **Validação Técnica (Engineer):** A relação é tecnicamente verdadeira?
           - Ex: 'ls' realmente lista arquivos? SIM.
           - Ex: 'Kernel' é um tipo de 'Mouse'? NÃO (REJECT).
           
        2. **Validação Pedagógica (Professor):** Existe dependência de aprendizado?
           - Se eu preciso aprender A para entender B -> Vote MODIFY e sugira **PREREQUISITE**.
           - Se A é uma ferramenta usada por B -> Vote MODIFY e sugira **USE**.
           - Se A compõe B -> Vote MODIFY e sugira **PART_OF**.
           - Se é apenas relacionado -> Vote APPROVE (RELATED_TO).
           
        Responda com veredito, tipo sugerido e razão breve.
        """
    )
    # CORREÇÃO: Usar .format() antes de invocar o modelo
    response = MODELS['expert'].invoke(prompt.format(source=state.source, target=state.target))
    
    content = response.content.upper()
    verdict = 'APPROVE'
    suggestion = None
    
    if 'REJECT' in content:
        verdict = 'REJECT'
    elif 'PREREQUISITE' in content:
        verdict = 'MODIFY'
        suggestion = 'PREREQUISITE'
    elif 'USE' in content:
        verdict = 'MODIFY'
        suggestion = 'USE'
    elif 'PART_OF' in content:
        verdict = 'MODIFY'
        suggestion = 'PART_OF'
    
    state.votes.append(AgentVote(agent="Expert", verdict=verdict, suggested_type=suggestion, reason=response.content[:200]))
    return state

# 3. THE ANALYST (Estrutura & Ontologia - GPT-4o)
def agent_analyst(state: EdgeState):
    if any(v.verdict == 'REJECT' for v in state.votes): return state

    prompt = ChatPromptTemplate.from_template(
        """Você é o ANALISTA ESTRUTURAL (Topólogo + Ontologista).
        Analise: {source} ({source_type}) -> {target} ({target_type}).
        
        ### CHECAGEM DE CONSISTÊNCIA:
        1. **Hierarquia de Tipos:**
           - Um 'Conceito Abstrato' não pode ser PART_OF um 'Comando'.
           - Hierarquia deve fluir do Geral -> Específico ou Componente -> Todo.
           
        2. **Topologia (Prevenção de Ciclos Locais):**
           - Verifique se a direção da seta faz sentido lógico.
           - Se estiver invertida (ex: 'Linux' PART_OF 'Kernel'), vote REJECT ou sugira inversão no comentário.
        
        Se tudo estiver logicamente consistente, vote APPROVE. Se houver erro categórico, vote REJECT.
        """
    )
    # CORREÇÃO: Usar .format() antes de invocar o modelo
    response = MODELS['analyst'].invoke(prompt.format(
        source=state.source, source_type=state.source_type,
        target=state.target, target_type=state.target_type
    ))
    
    verdict = 'REJECT' if 'REJECT' in response.content.upper() else 'APPROVE'
    state.votes.append(AgentVote(agent="Analyst", verdict=verdict, reason=response.content[:150]))
    return state

# 4. THE JUDGE (Decisor Final - GPT-4o)
class JudgeVerdict(BaseModel):
    final_action: Literal['KEEP', 'DISCARD']
    final_type: str
    rationale: str

def agent_judge(state: EdgeState):
    # Formata dossiê de votos
    votes_str = "\n".join([f"- {v.agent}: {v.verdict} ({v.suggested_type or 'N/A'}) -> {v.reason}" for v in state.votes])
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Você é o JUIZ SUPREMO do grafo SINKT. Sua palavra é final."),
        ("user", """Decida o destino desta aresta: {source} -> {target}
        
        VOTOS DO CONSELHO:
        {votes}
        
        ### REGRAS DE JULGAMENTO:
        1. **Veto Técnico:** Se o EXPERT ou CLEANER rejeitou, o veredito é DISCARD.
        2. **Tipificação:** Priorize o tipo sugerido pelo EXPERT (ex: PREREQUISITE) sobre tipos genéricos.
        3. **Segurança:** Na dúvida entre APPROVE e REJECT, prefira DISCARD para manter o grafo limpo.
        
        Retorne JSON compatível.
        {format_instructions}
        """)
    ])
    
    parser = PydanticOutputParser(pydantic_object=JudgeVerdict)
    try:
        # CORREÇÃO CRÍTICA: Invocar o modelo passando as mensagens formatadas pelo prompt, não um dicionário cru.
        # O dicionário vai para o .format(), e o resultado do .format() vai para o .invoke()
        
        formatted_messages = prompt.format_messages(
            source=state.source, 
            target=state.target, 
            votes=votes_str,
            format_instructions=parser.get_format_instructions()
        )
        
        response = MODELS['judge'].invoke(formatted_messages)
        verdict = parser.parse(response.content)
        
        state.final_verdict = verdict.final_action
        state.final_type = verdict.final_type
        state.final_reason = verdict.rationale
    except Exception as e:
        state.final_verdict = "DISCARD"
        state.final_reason = f"Erro Julgamento: {e}"
        # Adicionar print para debug no console também
        print(f"CRITICAL JUDGE ERROR: {e}")
        
    return state

## 3. Montagem do Workflow (LangGraph)

In [11]:
# Configurar Grafo (Otimizado 4 Agentes)
workflow = StateGraph(EdgeState)

# Adicionar Nós
workflow.add_node("cleaner", agent_cleaner)
workflow.add_node("expert", agent_expert)
workflow.add_node("analyst", agent_analyst)
workflow.add_node("judge", agent_judge)

# Definir Arestas (Fluxo)
workflow.set_entry_point("cleaner")

def cleaner_router(state):
    # Se cleaner rejeitar, vai direto pro fim (Judge apenas carimba)
    # Isso economiza execução dos agentes caros (Expert/Analyst)
    if any(v.verdict == 'REJECT' and v.agent == 'Cleaner' for v in state.votes):
        return "judge"
    return "expert"

workflow.add_conditional_edges("cleaner", cleaner_router)

# Fluxo Linear: Expert -> Analyst -> Judge
workflow.add_edge("expert", "analyst")
workflow.add_edge("analyst", "judge")
workflow.add_edge("judge", END)

app = workflow.compile()
print("✅ Workflow do Conselho (4 Agentes) compilado com sucesso.")

✅ Workflow do Conselho (4 Agentes) compilado com sucesso.


In [12]:
# --- EXECUÇÃO EM LOTE COM CHECKPOINT E LOGGING ---

print(f"⚖️ Iniciando Sessão do Conselho para {len(candidates)} candidatos...")

import os
import json
import datetime

CHECKPOINT_FILE = f"{OUTPUT_FOLDER}/checkpoint_edges.json"
LOG_FILE = f"{OUTPUT_FOLDER}/council_execution.log"
final_edges = []
processed_pairs = set()

# 1. Tentar carregar checkpoint existente
if os.path.exists(CHECKPOINT_FILE):
    try:
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            saved_data = json.load(f)
            final_edges = saved_data.get('edges', [])
            processed_pairs_all = set(saved_data.get('processed_pairs', []))
            
        print(f"🔄 Checkpoint carregado! {len(final_edges)} arestas aprovadas recuperadas.")
        print(f"⏩ Pulando {len(processed_pairs_all)} pares já processados anteriormente.")
    except Exception as e:
        print(f"⚠️ Erro ao ler checkpoint: {e}. Iniciando do zero.")
        processed_pairs_all = set()
else:
    processed_pairs_all = set()

# Função Helper para Log
def log_execution(pair, votes, verdict, reason):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as log:
        log.write(f"\n{'='*50}\n")
        log.write(f"[{timestamp}] PAR: {pair}\n")
        log.write(f"VEREDITO FINAL: {verdict}\n")
        log.write(f"RAZÃO: {reason}\n")
        log.write("-" * 20 + " VOTOS " + "-" * 20 + "\n")
        for v in votes:
            log.write(f" > {v.agent}: {v.verdict} (Sugestão: {v.suggested_type or 'N/A'}) | {v.reason[:100]}...\n")

# Processar em batch
candidates_to_process = [c for c in candidates if f"{c['source']}|{c['target']}" not in processed_pairs_all]

print(f"📌 Restam {len(candidates_to_process)} candidatos para processar.")

# Inicializar cabeçalho do log se arquivo novo
if not os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        f.write("--- LOG DE EXECUÇÃO DO CONSELHO ---\n")

for i, cand in enumerate(tqdm(candidates_to_process)):
    pair_id = f"{cand['source']}|{cand['target']}"
    
    initial_state = EdgeState(
        source=cand['source'],
        target=cand['target'],
        source_type=cand['source_type'],
        target_type=cand['target_type'],
        similarity_score=cand['score'],
        origin=cand['origin']
    )
    
    try:
        result = app.invoke(initial_state)
        
        # Logar execução detalhada
        log_execution(
            pair=pair_id, 
            votes=result['votes'], 
            verdict=result['final_verdict'], 
            reason=result['final_reason']
        )
        
        # Se aprovado, adiciona aos finais
        if result['final_verdict'] == 'KEEP':
            final_edges.append({
                "source": result['source'],
                "target": result['target'],
                "type": result['final_type'],
                "reason": result['final_reason'],
                "origin": result['origin']
            })
            
        processed_pairs_all.add(pair_id)
        
        # SALVAMENTO IMEDIATO (A CADA RETORNO)
        with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
            json.dump({
                "edges": final_edges,
                "processed_pairs": list(processed_pairs_all)
            }, f, indent=2, ensure_ascii=False)
                
    except Exception as e:
        print(f"❌ Erro no par {pair_id}: {e}")
        # Logar erro
        with open(LOG_FILE, "a", encoding="utf-8") as log:
             log.write(f"\n❌ ERRO NO PROCESSAMENTO DE {pair_id}: {e}\n")
        pass

print(f"\n✅ Processo Concluído.")
print(f"Arestas Aprovadas Totais: {len(final_edges)}")
print(f"📝 Log detalhado salvo em: {LOG_FILE}")

⚖️ Iniciando Sessão do Conselho para 252 candidatos...
📌 Restam 252 candidatos para processar.


100%|██████████| 252/252 [43:57<00:00, 10.47s/it]


✅ Processo Concluído.
Arestas Aprovadas Totais: 53
📝 Log detalhado salvo em: output/03_council_execution/council_execution.log


In [13]:
# --- CONSOLIDAÇÃO FINAL E SALVAMENTO ---

# Unir com as arestas originais (Assumimos que as originais da fase 2 são 'Ground Truth' parciais,
# mas idealmente deveriam passar pelo crivo também. Por hora, mantemos e adicionamos as novas)

all_relations = initial_relations + final_edges

# Construir Grafo Final para Métricas
G_final = nx.DiGraph()
for c in concepts:
    G_final.add_node(c['nome'], tipo=c['tipo'])

for r in all_relations:
    if G_final.has_node(r['source']) and G_final.has_node(r['target']):
        G_final.add_edge(r['source'], r['target'], type=r['type'])

print("="*40)
print("RELATÓRIO FINAL (MULTI-AGENT SWARM - OTIMIZADO)")
print("="*40)
print(f"Nós: {G_final.number_of_nodes()}")
print(f"Arestas: {G_final.number_of_edges()}")
print(f"Densidade: {nx.density(G_final):.5f}")

# Salvar
output_data = {
    "metadata": {
        "method": "Multi-Agent Council V2 (LangGraph Optimized)",
        "agents": ["Cleaner", "Expert", "Analyst", "Judge"],
        "models_used": list(MODELS.keys())
    },
    "concepts": concepts,
    "relations": all_relations
}

FINAL_FILE = f"{OUTPUT_FOLDER}/final_sinkt_graph_swarm.json"
with open(FINAL_FILE, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False)
    
print(f"💾 Grafo Salvo: {FINAL_FILE}")

RELATÓRIO FINAL (MULTI-AGENT SWARM - OTIMIZADO)
Nós: 216
Arestas: 260
Densidade: 0.00560
💾 Grafo Salvo: output/03_council_execution/final_sinkt_graph_swarm.json
